In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


In [2]:
(x_train, _), (_,_) = mnist.load_data()
print(x_train.shape)

(60000, 28, 28)


In [5]:
# img parameters
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

z_dim = 28*28 # 생성자의 입력으로 사용할 잡음 벡터의 크기

In [6]:
# making Generator

def build_generator(z_dim):
    model = Sequential()
    
    model.add(Dense(256 * 8 * 8, input_dim=z_dim)) # Dense 를 이용해 입력(Z)를 7 * 7 * 256 크기의 텐서로 변환
    model.add(Reshape((8,8,256)))
    
    model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same')) # 8*8*256 에서 16*16*128 크기의 텐서로 바꾸는 전치 합성곱 층
    
    model.add(BatchNormalization())
    
    model.add(LeakyReLU(alpha=0.01))
    
    model.add(Conv2DTranspose(64, kernel_size=3, strides=1, padding='same')) # 16*16*128 에서 16*16*64 텐서로 바꾸는 전치 합성곱 층
    
    model.add(BatchNormalization())
    
    model.add(LeakyReLU(alpha=0.01))
    
    model.add(Conv2DTranspose(3, kernel_size=3, strides=2, padding='same')) # strides : 2 16*16*64 에서 28*28*3 텐서로 변환
    
    model.add(Activation('tanh'))
    
    return model

In [7]:
# making Discriminator

def build_discriminator(img_shape):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape = img_shape, padding='same')) # img 받고 14*14*32 텐서로 변환
    
    model.add(LeakyReLU(alpha=0.01))
    
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same')) # 14*14*32에서 7*7*64 텐서로 변환
    
    model.add(LeakyReLU(alpha=0.01))
    
    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same')) # 7*7*64 에서 3*3*128 텐서로 변환
    
    model.add(LeakyReLU(alpha=0.01))
    
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    return model
    

In [8]:
def build_gan(G, D):
    model = Sequential()
    
    model.add(G)
    model.add(D)
    
    return model

D = build_discriminator(img_shape)
D.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

G = build_generator(z_dim)

D.trainable = False

gan = build_gan(G, D)
gan.compile(loss='binary_crossentropy', optimizer=Adam())


2022-09-04 17:10:02.865141: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-04 17:10:02.865223: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M2


ValueError: Exception encountered when calling layer "sequential" (type Sequential).

Input 0 of layer "conv2d" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (None, 32, 32, 3)

Call arguments received by layer "sequential" (type Sequential):
  • inputs=tf.Tensor(shape=(None, 32, 32, 3), dtype=float32)
  • training=False
  • mask=None

In [32]:
loss = []
acc = []
checkpoint = []

def train(iterations, batch_size, sample_interval):
    (x_train, _), (_,_) = cifar10.load_data()
    assert x_train.shape == (50000, 32, 32, 3)
    x_train = x_train / 127.5 - 1.0  # [0:255] 흑백 픽셀 값을 [-1:1]로 스케일 조정
    x_train = np.expand_dims(x_train, axis=3)
    
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for iteration in range(iterations):
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        imgs = x_train[idx]
        
        z = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = G.predict(z)
        
        d_loss_real = D.train_on_batch(imgs, real)
        d_loss_fake = D.train_on_batch(gen_imgs, fake)
        d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        z = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = G.predict(z)
        
        g_loss = gan.train_on_batch(z, real)
        
        if (iteration + 1) % sample_interval == 0:
            loss.append((d_loss, g_loss))
            acc.append(100.0 * accuracy)
            checkpoint.append(iteration + 1)
            
            print("{} D loss : {}, acc : {}, G loss : {}".format(iteration+1, d_loss, 100.0*accuracy, g_loss))
            
            sample_images(G)
            

In [33]:
def sample_images(G, image_grid_rows=4, image_grid_col=4):
    z = np.random.normal(0, 1, (image_grid_rows * image_grid_col, z_dim))
    
    gen_imgs = G.predict(z)
    
    gen_imgs = 0.5 + gen_imgs + 0.5 # 이미지 픽셀 값을 [0:1] 범위로 스케일 조정
    
    fig, axs = plt.subplots(image_grid_rows, image_grid_col, figsize=(4,4), sharey=True, sharex=True)
    cnt = 0 
    for i in range(image_grid_rows):
        for j in range(image_grid_col):
            axs[i,j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    plt.show()

In [34]:
iterations = 20000
batch_size = 128
sample_interval = 1000

train(iterations, batch_size, sample_interval)

ValueError: in user code:

    File "/Users/imjun-yeob/miniconda3/envs/tensorflow-dev/lib/python3.8/site-packages/keras/engine/training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "/Users/imjun-yeob/miniconda3/envs/tensorflow-dev/lib/python3.8/site-packages/keras/engine/training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/imjun-yeob/miniconda3/envs/tensorflow-dev/lib/python3.8/site-packages/keras/engine/training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/imjun-yeob/miniconda3/envs/tensorflow-dev/lib/python3.8/site-packages/keras/engine/training.py", line 1791, in predict_step
        return self(x, training=False)
    File "/Users/imjun-yeob/miniconda3/envs/tensorflow-dev/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from Non